# Dataset
1. E3-substrates-interactions file is downloaded from : http://ubibrowser.bio-it.cn/ubibrowser_v3/home/download
2. Collected_degrons file is downloaded from: http://degron.phasep.pro/download/ 

In [2]:
import pandas as pd

In [4]:
'''Convert E3-substrates-interactions text file to Excel file'''
# read in the text file using pandas
df = pd.read_csv('../datasets/E3-substrates-interactions.txt', delimiter='\t')

# write the dataframe to an Excel file
df.to_excel('../datasets/E3-substrates-interactions.xlsx', index=False)

# Preprocessing
1. Read the datasets
2. Remove the rows with nan or invalid IDs on Entry column in degrons, and unreviewd on their Status column
3. Remove the rows with nan or invalid IDs on both SwissProt AC (Substrate) and SwissProt AC (E3) columns
4. Merge them together on Entry to a final dataset
5. Keep needed columns: SwissProt AC (Substrate), SwissProt AC (E3), protein seq, start, end, deg_seq, E3_sequence
6. Explode the rows containing # in their E3
7. Add E3 sequences to the dataframe

In [9]:
degrons = pd.read_excel('../datasets/collected_degrons.xlsx')
interactions = pd.read_excel('../datasets/E3-substrates-interactions.xlsx')
E3_sequences = pd.read_excel('../datasets/uniprot-E3.xlsx')

#remove rows from both dataframes that have NaN or invalid ids
degrons = degrons[(degrons['Entry'].notnull()) & (degrons['Entry'] != '-')]
degrons = degrons[degrons['Status'] != 'unreviewd']

# Remove the rows with nan or invalid IDs on both SwissProt AC (Substrate) and SwissProt AC (E3) columns
interactions = interactions[(interactions['SwissProt AC (Substrate)'].notnull()) & (interactions['SwissProt AC (Substrate)'] != '-')]
interactions = interactions[(interactions['SwissProt AC (E3)'].notnull()) & (interactions['SwissProt AC (E3)'] != '-')]

# Merge them together on Entry to a final dataset
final = pd.merge(degrons, interactions, left_on='Entry', right_on='SwissProt AC (Substrate)')
final = pd.merge(final, E3_sequences, left_on='SwissProt AC (E3)', right_on='Entry')

# Keep needed columns: SwissProt AC (Substrate), SwissProt AC (E3), protein seq, start, end, deg_seq
final = final[['SwissProt AC (Substrate)', 'SwissProt AC (E3)', 'protein seq', 'start', 'end', 'deg_seq', 'E3_sequence']]
final.columns = ['substrate', 'E3', 'substrate protein seq', 'start', 'end', 'degron_seq', 'E3_sequence']
final = final.assign(E3=final['E3'].str.split('#')).explode('E3')
final = final.drop_duplicates(subset=['substrate', 'E3'], keep='first')

In [10]:
# Write the final dataset to an Excel file
final.to_excel('../datasets/final_dataset.xlsx', index=False)